In [14]:
from datasets import Dataset
from time import perf_counter
from autofaiss import build_index
import polars as pl
import numpy as np
from core_pro.ultilities import make_sync_folder, create_batch_index

In [15]:
path = make_sync_folder('dataset/item_matching')
file = path / 'data_sample_FMCG_clean.parquet'

df = pl.read_parquet(file)
df.head()

index,item_id,item_name,shop_id,shop_name,level1_global_be_category,level2_global_be_category,level3_global_be_category,cluster,description,images,image_url,item_name_clean,file_path
u32,i64,str,i64,str,str,str,str,str,str,str,str,str,str
0,19092271907,"""LINEABON K2D3 nhập khẩu châu â…",1053944724,"""ChiChi.Kids17""","""Mom & Baby""","""Baby Healthcare""","""Baby Vitamins & Supplements""","""FMCG""","""Nguồn gốc xuất xứ K2 D3 Lineab…","""vn-11134207-7ras8-m49p5xv9ecb3…","""http://f.shopee.vn/file/vn-111…","""lineabon k2d3 nhập khẩu châu â…","""/media/kevin/data_4t/dataset/i…"
1,17398587723,"""[MKB Gift] Bộ 3 Khăn ướt Moony…",63522286,"""Moony - Gian Hàng Chính Hãng""","""Mom & Baby""","""Bath & Body Care""","""Wipes""","""FMCG""","""[{""t"":""Bộ 3 khăn ướt Moony 50 …","""eb26f55f7359e0de333f0c34e2619f…","""http://f.shopee.vn/file/eb26f5…","""bộ 3 khăn ướt moony 50 miếng/g…","""/media/kevin/data_4t/dataset/i…"
2,19111299596,"""Bàn Chải Đánh Răng Lipzo Sensi…",170502615,"""Niva Lipzo Official Store""","""Health""","""Personal Care""","""Oral Care""","""FMCG""","""[{""t"":""BÀN CHẢI ĐÁNH RĂNG LIPZ…","""vn-11134207-7ras8-m2kwpjg8vb46…","""http://f.shopee.vn/file/vn-111…","""bàn chải đánh răng lipzo sensi…","""/media/kevin/data_4t/dataset/i…"
3,25583412760,"""Rong biển ăn liền Bibizan siêu…",119247917,"""King 21""","""Food & Beverages""","""Snacks""","""Seaweed""","""FMCG""","""NGÀY IN TRÊN BAO BÌ LÀ NGÀY SẢ…","""vn-11134207-7r98o-lxqfhxwp3h0b…","""http://f.shopee.vn/file/vn-111…","""rong biển ăn liền bibizan siêu…","""/media/kevin/data_4t/dataset/i…"
4,15930150764,"""(MUA 2 SON TẶNG 1 TÚI, 1 CUSHI…",181790483,"""Lam Thảo Cosmetics""","""Beauty""","""Makeup""","""Lips""","""FMCG""","""Son Tint Bóng Espoir Couture L…","""vn-11134207-7r98o-lyrll7tp71y9…","""http://f.shopee.vn/file/vn-111…","""son tint bóng espoir couture l…","""/media/kevin/data_4t/dataset/i…"


In [23]:
item = df['item_name_clean'].to_list()
path_array = path / 'tmp/array/bge/embed.npy'
embeddings = np.load(path_array)

In [20]:
start = perf_counter()

file_index = path / 'ip.index'
file_index_json = str(path / 'ip_index.json')
try:
    build_index(
        str(path_array.parent),
        index_path=str(file_index),
        index_infos_path=file_index_json,
        save_on_disk=True,
        metric_type='ip',
        verbose=30,
    )
    print(f'[BuildIndex] Time finished: {perf_counter() - start:,.2f}s')
except TypeError as e:
    print(f'[BuildIndex] Error: {e}')

end = perf_counter() - start
print(f'Total Index Time: {end:,.0f}s')

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: Object of type float32 is not JSON serializable

In [21]:
dataset = Dataset.from_polars(df)
dataset.load_faiss_index('vector', file_index)

In [27]:
start = perf_counter()

QUERY_SIZE = 20_000
run = create_batch_index(len(embeddings), QUERY_SIZE)
for i, val in run.items():
    start_idx, end_idx = val[0], val[-1]
    batch = embeddings[start_idx:end_idx]

    score, result = dataset.get_nearest_examples_batch(
        'vector',
        batch,
        k=10,
    )

    df_result = pl.DataFrame(result)
    dict_ = {f'score': [list(np.round(arr, 6)) for arr in score]}
    df_score = pl.DataFrame(dict_)
    break
end = perf_counter() - start
print(f'Total Query Time: {end:,.0f}s')

Total Query Time: 58s


In [28]:
df_result

shape: (19_999, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ index     ┆ item_id   ┆ item_name ┆ shop_id   ┆ … ┆ images    ┆ image_url ┆ item_name ┆ file_pat │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ _clean    ┆ h        │
│ list[i64] ┆ list[i64] ┆ list[str] ┆ list[i64] ┆   ┆ list[str] ┆ list[str] ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ list[str] ┆ list[str │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ [0,       ┆ [19092271 ┆ ["LINEABO ┆ [10539447 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["lineabo ┆ ["/media │
│ 30308, …  ┆ 907, 7820 ┆ N K2D3    ┆ 24,       ┆   ┆ 4207-7ras ┆ f.shopee. ┆ n k2d3    ┆ /kevin/d │
│ 30138]    ┆ 117814, … ┆ nhập khẩu ┆ 35162351, ┆   ┆ 8-m49p5xv ┆ vn/file/v ┆ nhập khẩu ┆ ata_4t/d │
│           ┆ 29…       ┆ châu…     ┆ … 50871…  ┆   ┆ 9ec…      ┆ n-1…      ┆ châu…     ┆ ataset…  │
│ [1,       ┆ [17398587 ┆ ["[MKB    ┆ [63522286 ┆ … ┆ ["eb26f55 ┆ ["http:// ┆ ["bộ 3    ┆ ["/media │
│ 34492, …  ┆ 723, 2631 ┆ Gift] Bộ  ┆ , 2442400 ┆   ┆ f7359e0de ┆ f.shopee. ┆ khăn ướt  ┆ /kevin/d │
│ 82240]    ┆ 6042157,  ┆ 3 Khăn    ┆ 66, …     ┆   ┆ 333f0c34e ┆ vn/file/e ┆ moony 50  ┆ ata_4t/d │
│           ┆ … 1…      ┆ ướt Moo…  ┆ 635219…   ┆   ┆ 261…      ┆ b26…      ┆ miếng…    ┆ ataset…  │
│ [2,       ┆ [19111299 ┆ ["Bàn     ┆ [17050261 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["bàn     ┆ ["/media │
│ 18563, …  ┆ 596, 2400 ┆ Chải Đánh ┆ 5, 170502 ┆   ┆ 4207-7ras ┆ f.shopee. ┆ chải đánh ┆ /kevin/d │
│ 81026]    ┆ 0347509,  ┆ Răng      ┆ 615, …    ┆   ┆ 8-m2kwpjg ┆ vn/file/v ┆ răng      ┆ ata_4t/d │
│           ┆ … 2…      ┆ Lipzo     ┆ 95286…    ┆   ┆ 8vb…      ┆ n-1…      ┆ lipzo     ┆ ataset…  │
│           ┆           ┆ Sen…      ┆           ┆   ┆           ┆           ┆ sen…      ┆          │
│ [3,       ┆ [25583412 ┆ ["Rong    ┆ [11924791 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["rong    ┆ ["/media │
│ 20230, …  ┆ 760, 2995 ┆ biển ăn   ┆ 7, 950574 ┆   ┆ 4207-7r98 ┆ f.shopee. ┆ biển ăn   ┆ /kevin/d │
│ 9469]     ┆ 4800382,  ┆ liền      ┆ 238, …    ┆   ┆ o-lxqfhxw ┆ vn/file/v ┆ liền      ┆ ata_4t/d │
│           ┆ … 1…      ┆ Bibizan   ┆ 10482…    ┆   ┆ p3h…      ┆ n-1…      ┆ bibizan   ┆ ataset…  │
│           ┆           ┆ si…       ┆           ┆   ┆           ┆           ┆ si…       ┆          │
│ [4, 4456, ┆ [15930150 ┆ ["(MUA 2  ┆ [18179048 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["son     ┆ ["/media │
│ … 91869]  ┆ 764, 1612 ┆ SON TẶNG  ┆ 3,        ┆   ┆ 4207-7r98 ┆ f.shopee. ┆ tint bóng ┆ /kevin/d │
│           ┆ 6980507,  ┆ 1 TÚI, 1  ┆ 4928507,  ┆   ┆ o-lyrll7t ┆ vn/file/v ┆ espoir    ┆ ata_4t/d │
│           ┆ … 2…      ┆ CUS…      ┆ …         ┆   ┆ p71…      ┆ n-1…      ┆ couture…  ┆ ataset…  │
│           ┆           ┆           ┆ 4308926…  ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ [19994,   ┆ [27606870 ┆ ["Khay    ┆ [13664052 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["khay    ┆ ["/media │
│ 50227, …  ┆ 103, 2495 ┆ Silicone  ┆ 1, 450868 ┆   ┆ 4207-7r98 ┆ f.shopee. ┆ silicone  ┆ /kevin/d │
│ 61945]    ┆ 7162511,  ┆ Trữ Đông  ┆ 882, …    ┆   ┆ o-lygnpf0 ┆ vn/file/v ┆ trữ đông  ┆ ata_4t/d │
│           ┆ … 2…      ┆ Đồ Ăn…    ┆ 17116…    ┆   ┆ 6cp…      ┆ n-1…      ┆ đồ ăn…    ┆ ataset…  │
│ [19995,   ┆ [25874037 ┆ ["Mắt mèo ┆ [27319058 ┆ … ┆ ["vn-1113 ┆ ["http:// ┆ ["mắt mèo ┆ ["/media │
│ 30140, …  ┆ 588, 2535 ┆ đỏ , Sơn  ┆ 7, 354289 ┆   ┆ 4207-7r98 ┆ f.shopee. ┆ đỏ , sơn  ┆ /kevin/d │
│ 77436]    ┆ 4405494,  ┆ thạch đỏ  ┆ 668, …    ┆   ┆ o-lu3ds5y ┆ vn/file/v ┆ thạch đỏ  ┆ ata_4t/d │
│           ┆ … 1…      ┆ rư…       ┆ 38338…    ┆   ┆ 5rr…      ┆ n-1…      ┆ rư…       ┆ ataset…  │
│ [19996,   ┆ [29257165 ┆ ["(Combo  ┆ [13